Could you please describe how you would approach this problem, what your plan would be, what considerations you’d make, what questions you’d need to find answers to, and what technologies you would select and why?

# Planning

First step would be to build a really dumb baseline that give sensible results.
This would allow us to start iterating and being able to track impact against baseline on the task. at every cycle taking into consideration
- validating assumptions with experiments
- business metric
- engineering resources
- data collection
- timelines


I built a dumb baseline in around 30 minutes in the other notebook in the repo. This are the outputs

![title](truecircle1.png)
![title](truecircle2.png)

# Purity by Weight: baseline

**Main idea**: object segmentation using a pretrained model on generic classes, use purity by area as a proxy estimate for purity by weight

The baseline was implemented with **Mask R-CNN** hoping that the bottle class would pop up, playing around with hyperparameter I managed to make it work. Assuming 
- the bottle class is plastic, every other class is not plastic
- assuming that area correlates with weight
With some quick post-processing (in the last section of the notebook) we can get an estimate of purity by weight that judging from the picture is sensible enough.

This was also an interesting test on how a generic model trained on vision would perform on the dataset


# Purity by Weight: architecture for next iterations

![](arch.png)

The main idea from the baseline is to have a **platic binary classifier** (is this object made of plastic) and a **weight predictor**, the purity by weight metric will be a weighted sum of this factors. We can choose a multi task architecture with a shared part that terminate on two heads. Here is an outline of the components of the architecture.

### INPUT PREPROCESSING
Images are really really ugly, stuff overlapping, dirt, background stuff. Looks like a lot of hanging fruit would be here, for instance some contrast based filtering, to remove noise from the background. 

## MASK R-CNN (w/ outputs)
This pretrained models are pretty good, as seen on the baseline they already start to understand something about the image. They should be of course fine tuned with custom labeled data. 200-300 images should be enough, with some bounding box annotator and custom classes. 

## Custom CNNs with cosine distance
This is the same thing as sentence similarity task, where you train two siamese networks with cosine distance. You basically take an anchor (plastic bottle) and a positive (other plastic bottle) and a negative (paper) and you make a cnn that start to detect features SPECFIC to our dataset. That's really important to complement the Mask R-CNN that has some super generic stuff. 

## Embeddings + two heads
Here get the fun part, you can do end to end training of your embeddings for the whole architecture if you manage to have labeled data. Now a fun way to generate this data is to generate them in the phyisical world. Re-create images similar to the one from production in a "lab" with accurate weighting of all the objects. You can now have perfect labels and virtually infinite data. Important: the data can't look like this http://wadaba.pcz.pl/#samples, this stuff is too clean, too idealistic, little to do with our dataset. That stuff might be good to train the custom CNN I mentioned above


### Literature review

Some folks tried to do visions on recyled plastic but their datasets are really clean! They don't look at all like our. This stuff can just be used for feature, but not like e2e models for our task

```

Application of Computer Vision Techniques for Segregation of PlasticWaste based on Resin Identification Code 
https://arxiv.org/abs/2011.07747, triplet loss interesting, probably not applicable to our case
    
https://github.com/garythung/trashnet too clean
    
Proença,Pedro&Simões,Pedro.(2020).TACO:TrashAnnotations in Context for Litter Detection.
    
WaDaBa database: too clean http://wadaba.pcz.pl/#samples
        
All these datasets are too clean, might try some transfer learning but would need tuning on our images.


Instead makes more sense to use Mask R-CNN
https://github.com/matterport/Mask_RCNN
```
        


# Other stuff

- You can use **any modern ML tech stack**, my preference would be to do the little architecture coding in pytorch, having some pipelines (probably airflow) to preprocess the images, serve the model through some AWS predictor service. The model shouldn't be too big so there are not too many problem on the infra side, we are just speaking about vision on a relatively small domain set. 
- **OCR!** that's a cool feature, would probably try to add as input to embedding some ad-hoc OCR model that extract text, that's probably really good to understand what the objects are
- One of the biggest unsolvable problems is objects being **stacked** on top of each other. Unless you x-ray or do fancy staff you are never going to actually know what's there.
- the question on **labeled data**, how much does it improve on the baseline? what kind of learning curves are we talking about? in case we need some humans to label data I would be really careful about this and really try hard with pre-trained stuff. On the other hand sounds an amazing idea to me to have a data generator in the lab that phyisically produce images, this is a super lucky situation where you can actually do that. If you really don't want to do it, you can still try to programmatically generate images, you take primitives (bottle, paper, bananas) with weight and bounding boxes, you might not even need GANs maybe some basic scripting and you can get some images that look pretty similar those in production.

Fun problem to work on!